# Apple Stock

### Introduction:

We are going to use Apple's stock price.


### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark = SparkSession.builder.appName("Apple_Stock").getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/09_Time_Series/Apple_Stock/appl_1980_2014.csv)

### Step 3. Assign it to a variable apple

In [3]:
url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/09_Time_Series/Apple_Stock/appl_1980_2014.csv'

from pyspark import SparkFiles
spark.sparkContext.addFile(url)


apple = spark.read.csv(SparkFiles.get("appl_1980_2014.csv"),header=True, inferSchema= True)


### Step 4.  Check out the type of the columns

In [4]:
apple.dtypes

[('Date', 'string'),
 ('Open', 'double'),
 ('High', 'double'),
 ('Low', 'double'),
 ('Close', 'double'),
 ('Volume', 'int'),
 ('Adj Close', 'double')]

### Step 5. Transform the Date column as a datetime type

### Step 6.  Set the date as the index

### Step 7.  Is there any duplicate dates?

### Step 8.  Ops...it seems the index is from the most recent date. Make the first entry the oldest date.

### Step 9. Get the last business day of each month

### Step 10.  What is the difference in days between the first day and the oldest

### Step 11.  How many months in the data we have?

### Step 12. Plot the 'Adj Close' value. Set the size of the figure to 13.5 x 9 inches

### BONUS: Create your own question and answer it.